In [122]:
import numpy as np

In [123]:
E = 3
tau = 1

In [124]:
x = np.array([1,2,3,4,5,6,7])

In [125]:
x.size

7

In [126]:
def retrieve_shadow_attractor(X, E=E):
    l = X.shape[0] - E + 1
    mx = np.empty((E, l))
    for i in range(E):
        # mx[E - i - 1] = X[i: i+l]
        mx[i] = X[i: i+l]

    return mx.T


In [127]:
mx = retrieve_shadow_attractor(x)
print(mx)

[[1. 2. 3.]
 [2. 3. 4.]
 [3. 4. 5.]
 [4. 5. 6.]
 [5. 6. 7.]]


In [128]:
def get_rate(mx, E=E):
    rate = np.empty((mx.shape[0], E - 1))
    for i in range(1, E):
        rate[:, i - 1] = mx[:, i] / mx[:, i-1] - 1

    return rate


In [129]:
rate = get_rate(mx)
print(rate)

[[1.         0.5       ]
 [0.5        0.33333333]
 [0.33333333 0.25      ]
 [0.25       0.2       ]
 [0.2        0.16666667]]


In [130]:
def get_dist(mx, E=E):
    l = mx.shape[0]
    dist = np.empty((l, l))
    dist.fill(float('inf'))
    for i in range(l):
        for j in range(i + 1, l):
            dist[i, j] = np.linalg.norm(mx[i] - mx[j])
            dist[j, i] = dist[i, j]

    return dist


In [131]:
dist = get_dist(mx)
print(dist)

[[       inf 1.73205081 3.46410162 5.19615242 6.92820323]
 [1.73205081        inf 1.73205081 3.46410162 5.19615242]
 [3.46410162 1.73205081        inf 1.73205081 3.46410162]
 [5.19615242 3.46410162 1.73205081        inf 1.73205081]
 [6.92820323 5.19615242 3.46410162 1.73205081        inf]]


In [132]:
def get_nn_weight(dist, no=E+1):
    nn_dist = np.sort(dist, axis=1)[:, :no]
    nn_index = np.argsort(dist, axis=1)[:, : no]
    nn_w = np.empty(nn_dist.shape)

    for i in range(nn_w.shape[0]):
        row = nn_dist[i]
        e_row = np.exp(np.max(row) - row)
        nn_w[i] = e_row / np.sum(e_row)

    return nn_w, nn_index


In [133]:
nn_w, nn_index = get_nn_weight(dist)
print(nn_w, nn_index)

[[0.823886   0.14576291 0.02578855 0.00456254]
 [0.45285295 0.45285295 0.08011929 0.0141748 ]
 [0.42483728 0.42483728 0.07516272 0.07516272]
 [0.45285295 0.45285295 0.08011929 0.0141748 ]
 [0.823886   0.14576291 0.02578855 0.00456254]] [[1 2 3 4]
 [0 2 3 4]
 [1 3 0 4]
 [2 4 1 0]
 [3 2 1 0]]


In [134]:
def get_signature(nn_w, rate, indexes):
    l = nn_w.shape[0]
    step = rate.shape[1]
    S = np.empty((l, step))
    for t in range(l):
        S[t] = np.dot(nn_w[t], rate[indexes[t]])
    
    return S


In [135]:
get_signature(nn_w, rate, nn_index)

array([[0.46789028, 0.31698753],
       [0.62666872, 0.35802604],
       [0.40882323, 0.27668836],
       [0.29575602, 0.22248256],
       [0.27201595, 0.21209538]])